# Filings Daily Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
import functools
from config import Config

%load_ext sql
%config SqlMagic.displaylimit = 5

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
%sql $Config.SQLALCHEMY_DATABASE_URI

monthly total before running time.

In [ ]:
%%sql icbc_data <<
select 
 CASE WHEN state='ACTIVE' THEN '1'
            WHEN state='HISTORICAL' THEN '2'
 END 
|| 
entity_type
||' '
||substr(identifier,3,7)
|| 
CASE WHEN legal_name IS NULL THEN '' 
     ELSE upper(legal_name)
END
from legal_entities
where entity_type in ('SP','GP');

In [ ]:
icbc_data_filename = os.path.join(os.getcwd(), r'data/') +'from_openshift.txt'
   
with open(icbc_data_filename, 'w') as f:
    if not icbc_data.DataFrame().empty:
        df=icbc_data.DataFrame()
        # folling for looper change the string output from right alignment to left alignment.
        formatters = {}        
        for li in list(df.columns):
            max = df[li].str.len().max()
            form = "{{:<{}s}}".format(max)
            formatters[li] = functools.partial(str.format, form)

        dfAsString = df.to_string(formatters=formatters, header=False, index=False) 
        f.write(dfAsString)